In [2]:
#importing the necessary python libraries


In [3]:
import os
from sqlalchemy import create_engine


# Construct the connection URL
db_url = f"sqlite:///{os.path.abspath('/home/muregz/data_analysis/song_sales/chinook.db')}"

# Set the DATABASE_URL environment variable
os.environ['DATABASE_URL'] = db_url

# Create a database engine
engine = create_engine(os.environ['DATABASE_URL'])


In [4]:
#configuring mysql into jupyter notebook
%load_ext sql
%config SqlMagic.autopandas=True

**1.Which tracks generated the most revenue? which album? which genre?**

In [5]:
%%sql
SELECT tracks.Name AS `Song`, albums.Title AS `Album`, genres.Name 
AS `Genre`, SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS Revenue
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN genres ON tracks.GenreId = genres.GenreId
JOIN invoice_items ON tracks.TrackId = invoice_items.TrackId
GROUP BY tracks.TrackId
ORDER BY revenue DESC
LIMIT 5;


Done.


,Song,Album,Genre,Revenue
0,The Woman King,"Battlestar Galactica, Season 3",Science Fiction,3.98
1,The Fix,"Heroes, Season 1",Drama,3.98
2,Walkabout,"Lost, Season 1",TV Shows,3.98
3,Hot Girl,"The Office, Season 1",TV Shows,3.98
4,Gay Witch Hunt,"The Office, Season 3",TV Shows,3.98


**2.Which tracks appeared in the most playlists?How many playlists did they appear in?**

In [6]:
%%sql
SELECT tracks.Name, COUNT(*) AS `Number of Appearances`
FROM playlist_track
JOIN tracks
ON playlist_track.TrackId = tracks.TrackId
GROUP BY playlist_track.TrackId
ORDER BY COUNT(*) DESC
LIMIT 10;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,Name,Number of Appearances
0,Intoitus: Adorate Deum,5
1,"Miserere mei, Deus",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,"The Messiah: Behold, I Tell You a Mystery... T...",5
5,Solomon HWV 67: The Arrival of the Queen of Sheba,5
6,Symphony No.5 in C Minor: I. Allegro con brio,5
7,Ave Maria,5
8,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate""",5
9,Die Walküre: The Ride of the Valkyries,5


**3.Which countries have the highest sales revenue? What percentage of total revenue does each country make up?**

In [7]:
%%sql
SELECT invoices.BillingCountry AS `Country`, SUM(invoices.Total) AS Revenue,
ROUND((SUM(invoices.Total) * 100)/SUM(SUM(invoices.Total)) OVER(), 2) AS `Percentage of Total Revenue`
FROM invoices
GROUP BY invoices.BillingCountry
ORDER BY Revenue DESC
LIMIT 10;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,Country,Revenue,Percentage of Total Revenue
0,USA,523.06,22.46
1,Canada,303.96,13.05
2,France,195.10,8.38
3,Brazil,190.10,8.16
4,Germany,156.48,6.72
5,United Kingdom,112.86,4.85
6,Czech Republic,90.24,3.88
7,Portugal,77.24,3.32
8,India,75.26,3.23
9,Chile,46.62,2.00


**4.How many customers did each employee support, what is the average revenue for each sale, and what is their total sale?**

In [8]:
%%sql
SELECT employees.FirstName, employees.LastName,
COUNT(customers.SupportRepId) AS `Customers Supported`,
SUM(invoices.Total) AS Revenue,
ROUND((SUM(invoices.Total) * 100)/SUM(SUM(invoices.Total)) OVER(), 2) AS `Percentage of Total Revenue`
FROM employees
JOIN customers ON employees.EmployeeId = customers.SupportRepId
JOIN invoices ON customers.SupportRepId = invoices.CustomerId
GROUP BY 1
ORDER BY Revenue DESC
LIMIT 10;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,FirstName,LastName,Customers Supported,Revenue,Percentage of Total Revenue
0,Jane,Peacock,147,832.02,35.32
1,Margaret,Park,140,792.40,33.64
2,Steve,Johnson,126,731.16,31.04


**5. Do longer or shorter length albums tend to generate more revenue?**

In [ ]:
%%sql
--adding a column to the tracks table to store track length in minutes
ALTER TABLE tracks ADD duration_in_minutes INT;
UPDATE TABLE tracks SET duration_in_minutes = REAL;


In [21]:
%%sql
--adding data into the column added
UPDATE tracks SET duration_in_minutes = ROUND(CAST(Milliseconds AS REAL) / 60000,  2);


 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
3503 rows affected.


""


In [27]:
%%sql
WITH sales AS(
    SELECT invoices.InvoiceId, SUM(invoices.Total) AS Revenue
            ,invoice_items.TrackId AS Tracks
    FROM invoices
    JOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId
    GROUP BY 1, 3
)
SELECT albums.Title, SUM(duration_in_minutes) AS length --length is in minutes
        ,SUM(sales.Revenue) AS `Total Revenue`
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN sales ON tracks.AlbumId = sales.Tracks
GROUP BY 1, albums.AlbumId
ORDER BY SUM(sales.Revenue) DESC
LIMIT 20;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,Title,length,Total Revenue
0,"Lost, Season 3",2355.54,386.10
1,Roda De Funk,138.86,364.32
2,The Essential Miles Davis [Disc 1],153.24,296.01
3,International Superhits,60.77,291.06
4,Motley Crue Greatest Hits,147.86,269.28
5,Live After Death,97.14,249.48
6,Os Cães Ladram Mas A Caravana Não Pára,98.92,237.60
7,Are You Experienced?,60.29,235.62
8,Blood Sugar Sex Magik,73.95,235.62
9,The Battle Rages On,100.58,227.70
